In [1]:
from skimage.measure import marching_cubes
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [13]:
pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 4.7 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import numpy as np
import cv2
import nibabel as nib
import pandas as pd
#%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback 
from skimage.transform import resize
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import scipy.ndimage

ModuleNotFoundError: No module named 'pandas'

In [8]:
# Function to load NIfTI files with memory mapping
def load_nifti_memmap(file_path):
    img = nib.load(file_path)
    data = img.get_fdata(dtype=np.float32, caching='unchanged')  # Memory-mapped array
    affine = img.affine
    header = img.header
    return data, affine, header

# Generator function to load data in batches
def data_generator(file_list, data_path, mask_path, batch_size, target_shape=None):
    while True:
        np.random.shuffle(file_list)
        for start in range(0, len(file_list), batch_size):
            end = min(start + batch_size, len(file_list))
            batch_files = file_list[start:end]
            
            X_batch = []
            y_batch = []
            
            for filename in batch_files:
                img_path = os.path.join(data_path, filename)
                corresponding_mask_path = os.path.join(mask_path, filename)
                
                image, _, _ = load_nifti_memmap(img_path)
                mask, _, _ = load_nifti_memmap(corresponding_mask_path)
                
                # Ensure image and mask have the same shape (and possibly resize if needed)
                if target_shape:
                    image = resize_volume(image, target_shape)
                    mask = resize_volume(mask, target_shape)
                
                X_batch.append(image)
                y_batch.append(mask)
            
            X_batch = np.array(X_batch)[..., np.newaxis]  # Adding channel dimension
            y_batch = np.array(y_batch)[..., np.newaxis]  # Adding channel dimension
            
            yield X_batch, y_batch

# Function to resize volumes (if needed)
def resize_volume(img, target_shape):
    current_shape = img.shape
    if current_shape == target_shape:
        return img
    # Example: using scipy for interpolation
    resized_img = scipy.ndimage.zoom(img, (target_shape[0]/current_shape[0], target_shape[1]/current_shape[1], target_shape[2]/current_shape[2]), order=3)
    return resized_img

def resize_image(image, target_shape):
    # Resize the image to match the target shape (height, width)
    return resize(image, target_shape, preserve_range=True, anti_aliasing=True)

def pad_or_crop_volume(volume, target_shape):
    current_shape = volume.shape
    
    # Calculate padding width for each dimension
    pad_width = [(0, max(target_shape[i] - current_shape[i], 0)) for i in range(3)]
    
    # Pad the volume to the target shape
    volume = np.pad(volume, pad_width, mode='constant', constant_values=0)
    
    # Calculate cropping dimensions for each dimension
    crop_start = [(volume.shape[i] - target_shape[i]) // 2 for i in range(3)]
    crop_end = [crop_start[i] + target_shape[i] for i in range(3)]
    
    # Crop the volume to the target shape
    slices = [slice(crop_start[i], crop_end[i]) for i in range(3)]
    volume = volume[slices[0], slices[1], slices[2]]
    
    return volume

def calculate_volume(mask, voxel_volume):
    # Calculate the volume based on the mask
    return np.sum(mask) * voxel_volume
    
def unet_3d(input_shape):
    inputs = Input(input_shape)
    
    # Downsampling
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)  # Added layer
    c1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c1)  # Added layer
    p1 = MaxPooling3D((2, 2, 2))(c1)
    
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(p1)
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)  # Added layer
    c2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c2)  # Added layer
    p2 = MaxPooling3D((2, 2, 2))(c2)
    
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(p2)
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)  # Added layer
    c3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c3)  # Added layer
    p3 = MaxPooling3D((2, 2, 2))(c3)
    
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(p3)
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)  # Added layer
    c4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(c4)  # Added layer
    
    # Upsampling
    u5 = UpSampling3D((2, 2, 2))(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(u5)
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c5)
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c5)  # Added layer
    c5 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(c5)  # Added layer
    
    u6 = UpSampling3D((2, 2, 2))(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(u6)
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c6)
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c6)  # Added layer
    c6 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(c6)  # Added layer
    
    u7 = UpSampling3D((2, 2, 2))(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(u7)
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c7)
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c7)  # Added layer
    c7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(c7)  # Added layer

    outputs = Conv3D(1, (1, 1, 1), activation='sigmoid')(c7)
    
    return Model(inputs, outputs)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model
    
    model = Model(inputs, outputs)
    #model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', dice_coefficient, jaccard_index])
    return model

# Dice coefficient
def dice_coefficient(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + 1)

# Jaccard index
def jaccard_index(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (intersection + 1) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection + 1)

# Example paths (update these to your dataset paths)
#data_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Train/imageTr'  # Update with the correct path
#mask_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Train/LabelTr'  # Update with the correct path

# Paths for data and masks
train_data_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Train/imageTr'
train_mask_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Train/LabelTr'

val_data_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Val/imageVal'
val_mask_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Val/LabelVal'

test_data_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Test/imageTs'
test_mask_path = r'/home/icmr/Documents/DATASET/MSD_LIVER/Portal/Test/LabelTs'
'''
# Verify paths exist
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data path not found: {data_path}")
if not os.path.exists(mask_path):
    raise FileNotFoundError(f"Mask path not found: {mask_path}")
'''
# Verify paths exist
for path in [train_data_path, train_mask_path, val_data_path, val_mask_path, test_data_path, test_mask_path]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Path not found: {path}")


# List of files
#file_list = [filename for filename in os.listdir(data_path) if filename.endswith('.nii.gz')]
#file_list_test = 
# Split the dataset
#train_val_files, test_files = train_test_split(file_list, test_size=0.2, random_state=42)
#train_files, val_files = train_test_split(train_val_files, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# File lists for each dataset
train_files = [filename for filename in os.listdir(train_data_path) if filename.endswith('.nii.gz')]
val_files = [filename for filename in os.listdir(val_data_path) if filename.endswith('.nii.gz')]
test_files = [filename for filename in os.listdir(test_data_path) if filename.endswith('.nii.gz')]


# Define and compile the model
input_shape = (128, 128, 64, 1)  # Example input shape, adjust accordingly
model = unet_3d(input_shape)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', dice_coefficient, jaccard_index])
  
# Define callbacks
#checkpoint = ModelCheckpoint('unet3d_best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define batch size
batch_size = 2
'''
# Create data generators
train_generator = data_generator(train_files, data_path, mask_path, batch_size, target_shape=input_shape[:3])
val_generator = data_generator(val_files, data_path, mask_path, batch_size, target_shape=input_shape[:3])
test_generator = data_generator(test_files, data_path, mask_path, batch_size, target_shape=input_shape[:3])
'''
# Data generators
train_generator = data_generator(train_files, train_data_path, train_mask_path, batch_size, target_shape=input_shape[:3])
val_generator = data_generator(val_files, val_data_path, val_mask_path, batch_size, target_shape=input_shape[:3])
test_generator = data_generator(test_files, test_data_path, test_mask_path, batch_size, target_shape=input_shape[:3])


# Calculate steps per epoch
steps_per_epoch = len(train_files) // batch_size
validation_steps = len(val_files) // batch_size
test_steps = len(test_files) // batch_size

# Train the model
#history = model.fit(train_generator, validation_data=val_generator, epochs=200, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, callbacks=[checkpoint])

# Save the model
model.save('unet3d_model.h5')

# Define custom callback for metrics
class MetricsLogger(Callback):
    def __init__(self, val_generator, val_steps, test_generator, test_steps):
        super().__init__()
        self.val_generator = val_generator
        self.val_steps = val_steps
        self.test_generator = test_generator
        self.test_steps = test_steps
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        # Evaluate on validation set
        val_loss, val_accuracy, val_dice, val_jaccard = self.model.evaluate(
            self.val_generator, steps=self.val_steps, verbose=0
        )
        # Evaluate on test set
        test_loss, test_accuracy, test_dice, test_jaccard = self.model.evaluate(
            self.test_generator, steps=self.test_steps, verbose=0
        )
        # Log metrics
        self.logs.append({
            'epoch': epoch + 1,
            'train_loss': logs.get('loss'),
            'train_accuracy': logs.get('accuracy'),
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
            'val_dice': val_dice,
            'val_jaccard': val_jaccard,
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
            'test_dice': test_dice,
            'test_jaccard': test_jaccard,
        })
        print(f"Epoch {epoch + 1}: "
              f"Train Loss: {logs.get('loss')}, Train Accuracy: {logs.get('accuracy')}, "
              f"Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, "
              f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Define callbacks
checkpoint = ModelCheckpoint('unet3d_best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
metrics_logger = MetricsLogger(val_generator, validation_steps, test_generator, test_steps)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=200,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint, metrics_logger]
)

# Final evaluation on test set
final_test_loss, final_test_accuracy, final_test_dice, final_test_jaccard = model.evaluate(
    test_generator, steps=test_steps
)
print("\nFinal Test Metrics:")
print(f"Test Loss: {final_test_loss}")
print(f"Test Accuracy: {final_test_accuracy}")
print(f"Test Dice Coefficient: {final_test_dice}")
print(f"Test Jaccard Index: {final_test_jaccard}")

# Save metrics logs
import pandas as pd
metrics_df = pd.DataFrame(metrics_logger.logs)
metrics_df.to_csv('metrics_log.csv', index=False)

# Display metrics as a table
print("\nMetrics per Epoch:")
print(metrics_df)


Epoch 1/200


2025-01-02 13:33:14.792684: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2025-01-02 13:33:15.178452: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-01-02 13:33:15.899759: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-01-02 13:33:23.068544: I external/local_xla/xla/service/service.cc:168] XLA service 0x7e26f62fb720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-02 13:33:23.068580: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-01-02 13:33:23.086843: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735805003.249205   26176 device_compiler.h:186] 

21/21 [==============================] - 1057s 51s/step - loss: 297.0812 - accuracy: 0.7387 - dice_coefficient: 0.0842 - jaccard_index: 0.0461 - val_loss: 0.8529 - val_accuracy: 0.9196 - val_dice_coefficient: 2.5050e-05 - val_jaccard_index: 2.5042e-05
Epoch 2/200
21/21 [==============================] - 1034s 52s/step - loss: 0.5386 - accuracy: 0.8078 - dice_coefficient: 0.0377 - jaccard_index: 0.0201 - val_loss: 0.0967 - val_accuracy: 0.9207 - val_dice_coefficient: 0.0928 - val_jaccard_index: 0.0487
Epoch 3/200
21/21 [==============================] - 970s 48s/step - loss: 0.1184 - accuracy: 0.8106 - dice_coefficient: 0.1504 - jaccard_index: 0.0824 - val_loss: 0.0842 - val_accuracy: 0.9163 - val_dice_coefficient: 0.1452 - val_jaccard_index: 0.0784
Epoch 4/200
21/21 [==============================] - 951s 48s/step - loss: 0.1306 - accuracy: 0.8110 - dice_coefficient: 0.1760 - jaccard_index: 0.0975 - val_loss: 0.0762 - val_accuracy: 0.9214 - val_dice_coefficient: 0.1697 - val_jaccard_in

ModuleNotFoundError: No module named 'pandas'

In [1]:
def display_results(model, file_list, test_data_path, test_mask_path):
    target_shape = (128, 128, 64)  # Fixed shape for all volumes
    
    # Iterate through all files in the file list
    for i in range(len(file_list)):
        # Load image and mask
        image_file_path = os.path.join(test_data_path, file_list[i])
        mask_file_path = os.path.join(test_mask_path, file_list[i])
        
        print(f"Processing file {i+1}/{len(file_list)}: {file_list[i]}")
        
        try:
            # Load image and mask data from .nii files
            image = nib.load(image_file_path).get_fdata(dtype=np.float32)
            mask = nib.load(mask_file_path).get_fdata(dtype=np.float32)
            
            # Resize image and mask to expected shape
            image = resize_image(image, target_shape[:2])
            mask = resize_image(mask, target_shape[:2])
            
            # Pad or crop image and mask to match target depth
            image = pad_or_crop_volume(image, target_shape)
            mask = pad_or_crop_volume(mask, target_shape)
            
            # Preprocess image and mask
            image = image[..., np.newaxis]
            mask = mask[..., np.newaxis]

            # Get prediction
            prediction = model.predict(np.expand_dims(image, axis=0))[0, ..., 0]
            
            # Visualize each slice of the image
            for z in range(target_shape[2]):
                fig, axs = plt.subplots(1, 3, figsize=(15, 5))
                
                # Subplot 1: Original Image Slice
                axs[0].imshow(image[:, :, z, 0], cmap='gray')
                axs[0].set_title(f"Image Slice {z}")
                axs[0].axis('off')

                # Subplot 2: Ground Truth Mask Slice
                axs[1].imshow(mask[:, :, z, 0], cmap='gray')
                axs[1].set_title(f"Mask Slice {z}")
                axs[1].axis('off')

                # Subplot 3: Prediction Slice
                axs[2].imshow(prediction[:, :, z], cmap='gray')
                axs[2].set_title(f"Prediction Slice {z}")
                axs[2].axis('off')

                plt.tight_layout()
                plt.show()
        
        except Exception as e:
            print(f"Error processing file {file_list[i]}: {e}")

# Call the function 
display_results(model, test_data_path, test_mask_path)



NameError: name 'model' is not defined